In [1]:
import pandas as pd
import numpy as np

In [2]:
telco_raw = pd.read_csv('../DATASETS/part4/telco.csv')
print(telco_raw.head())

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

In [3]:
#telco_raw['TotalCharges'] = telco_raw['TotalCharges'].str.replace(' ', '0')
telco_raw['TotalCharges'] = telco_raw['TotalCharges'].replace(r'^\s*$', np.nan, regex=True)

In [4]:
telco_raw['TotalCharges'] =telco_raw['TotalCharges'].astype(float)

In [5]:
telco_raw.dropna(inplace=True)

In [6]:
# Store customerID and Churn column names
custid = ['customerID']
target = ['Churn']

# Store categorical column names
categorical = telco_raw.nunique()[telco_raw.nunique() < 5].keys().tolist()

# Remove target from the list of categorical variables
categorical.remove(target[0])

# Store numerical column names
numerical = [x for x in telco_raw.columns if x not in custid + target + categorical]

In [7]:
print(categorical)

['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


In [8]:
print(numerical)

['tenure', 'MonthlyCharges', 'TotalCharges']


In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
# Perform one-hot encoding to categorical variables 
telco_raw = pd.get_dummies(data = telco_raw, columns = categorical, drop_first=True)
print(telco_raw.head())

   customerID  tenure  MonthlyCharges  TotalCharges Churn  gender_Male  \
0  7590-VHVEG       1           29.85         29.85    No            0   
1  5575-GNVDE      34           56.95       1889.50    No            1   
2  3668-QPYBK       2           53.85        108.15   Yes            1   
3  7795-CFOCW      45           42.30       1840.75    No            1   
4  9237-HQITU       2           70.70        151.65   Yes            0   

   SeniorCitizen_1  Partner_Yes  Dependents_Yes  PhoneService_Yes  ...  \
0                0            1               0                 0  ...   
1                0            0               0                 1  ...   
2                0            0               0                 1  ...   
3                0            0               0                 0  ...   
4                0            0               0                 1  ...   

   StreamingTV_No internet service  StreamingTV_Yes  \
0                                0                0   


In [11]:
# Initialize StandardScaler instance
scaler = StandardScaler()

# Fit and transform the scaler on numerical columns
scaled_numerical = scaler.fit_transform(telco_raw[numerical])

In [12]:
# Build a DataFrame from scaled_numerical
scaled_numerical = pd.DataFrame(scaled_numerical, columns=numerical)

In [13]:
scaled_numerical.head()

,tenure,MonthlyCharges,TotalCharges
0,-1.280248,-1.161694,-0.994194
1,0.064303,-0.260878,-0.173740
2,-1.239504,-0.363923,-0.959649
3,0.512486,-0.747850,-0.195248
4,-1.239504,0.196178,-0.940457


In [14]:
telco_raw = pd.get_dummies(data = telco_raw, columns = target, drop_first=True)


In [15]:
telcom = telco_raw
telcom[numerical] = scaled_numerical
telcom.dropna(inplace = True)
print(telcom.head())

   customerID    tenure  MonthlyCharges  TotalCharges  gender_Male  \
0  7590-VHVEG -1.280248       -1.161694     -0.994194            0   
1  5575-GNVDE  0.064303       -0.260878     -0.173740            1   
2  3668-QPYBK -1.239504       -0.363923     -0.959649            1   
3  7795-CFOCW  0.512486       -0.747850     -0.195248            1   
4  9237-HQITU -1.239504        0.196178     -0.940457            0   

   SeniorCitizen_1  Partner_Yes  Dependents_Yes  PhoneService_Yes  \
0                0            1               0                 0   
1                0            0               0                 1   
2                0            0               0                 1   
3                0            0               0                 0   
4                0            0               0                 1   

   MultipleLines_No phone service  ...  StreamingTV_Yes  \
0                               1  ...                0   
1                               0  ...        

# churn prediction

In [16]:
telcom.rename(columns={'Churn_Yes':'Churn'}, inplace=True)

# Print the unique Churn values
print(set(telcom['Churn']))

# Calculate the ratio size of each churn group
print(telcom.groupby(['Churn']).size() / telcom.shape[0] * 100)

# Import the function for splitting data to train and test
from sklearn.model_selection import train_test_split

# Split the data into train and test
train, test = train_test_split(telcom, test_size = .25)

{0, 1}
Churn
0    73.422589
1    26.577411
dtype: float64


general consensus when assessing class ration is at least one class constitute 5% of data

In [17]:
# Store column names from `telcom` excluding target variable and customer ID
cols = [col for col in telcom.columns if col not in custid + target]

# Extract training features
train_X = train[cols].values

# Extract training target
train_Y = train[target].values.ravel()

# Extract testing features
test_X = test[cols].values

# Extract testing target
test_Y = test[target].values.ravel()

In [18]:
type(train_Y)

numpy.ndarray

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score

In [20]:
logreg = LogisticRegression()

logreg.fit(train_X, train_Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
pred_test_Y = logreg.predict(test_X)

# Calculate accuracy score on testing data
test_accuracy = accuracy_score(test_Y, pred_test_Y)

# Print test accuracy score rounded to 4 decimals
print('Test accuracy:', round(test_accuracy, 4))

Test accuracy: 0.7904


In [22]:
# Initialize logistic regression instance 
logreg = LogisticRegression(penalty='l1', C=0.025, solver='liblinear')

# Fit the model on training data
logreg.fit(train_X, train_Y)

# Predict churn values on test data
pred_test_Y = logreg.predict(test_X)

# Print the accuracy score on test data
print('Test accuracy:a', round(accuracy_score(test_Y, pred_test_Y), 4))

Test accuracy:a 0.7825


In [23]:
C = [1, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01, 0.005, 0.0025]
col_names =['C', 'Non-Zero Coeffs', 'Recall']
l1_metrics = np.zeros((len(C),3))
l1_metrics[:,0] = C

In [24]:
# Run a for loop over the range of C list length
for index in range(0, len(C)):
  # Initialize and fit Logistic Regression with the C candidate
  logreg = LogisticRegression(penalty='l1', C=C[index], solver='liblinear')
  logreg.fit(train_X, train_Y)
  # Predict churn on the testing data
  pred_test_Y = logreg.predict(test_X)
  # Create non-zero count and recall score columns
  l1_metrics[index,1] = np.count_nonzero(logreg.coef_)
  l1_metrics[index,2] = recall_score(test_Y, pred_test_Y)

# Name the columns and print the array as pandas DataFrame
col_names = ['C','Non-Zero Coeffs','Recall']
print(pd.DataFrame(l1_metrics, columns=col_names))

        C  Non-Zero Coeffs    Recall
0  1.0000             25.0  0.535865
1  0.5000             26.0  0.535865
2  0.2500             25.0  0.533755
3  0.1000             22.0  0.525316
4  0.0500             22.0  0.474684
5  0.0250             16.0  0.434599
6  0.0100             10.0  0.223629
7  0.0050              8.0  0.000000
8  0.0025              0.0  0.000000


# decission tree

In [25]:
from sklearn import tree

In [26]:
# Initialize decision tree classifier
mytree = tree.DecisionTreeClassifier()

# Fit the decision tree on training data
mytree.fit(train_X, train_Y)

# Predict churn labels on testing data
pred_test_Y = mytree.predict(test_X)

# Calculate accuracy score on testing data
test_accuracy = accuracy_score(test_Y, pred_test_Y)

# Print test accuracy
print('Test accuracy:', round(test_accuracy, 4))

Test accuracy: 0.7124


In [27]:
depth_list = list(range(2,15)) 
depth_tuning = np.zeros((len(depth_list), 2)) 
depth_tuning[:,0] = depth_list

# Run a for loop over the range of depth list length
for index in range(0, len(depth_list)):
    # Initialize and fit decision tree with the `max_depth` candidate
    mytree = tree.DecisionTreeClassifier(max_depth=depth_list[index])
    mytree.fit(train_X, train_Y)
    # Predict churn on the testing data
    pred_test_Y = mytree.predict(test_X)
    # Calculate the recall score 
    depth_tuning[index,1] = recall_score(test_Y, pred_test_Y)

# Name the columns and print the array as pandas DataFrame
col_names = ['Max_Depth','Recall']
print(pd.DataFrame(depth_tuning, columns=col_names))

    Max_Depth    Recall
0         2.0  0.525316
1         3.0  0.518987
2         4.0  0.417722
3         5.0  0.424051
4         6.0  0.364979
5         7.0  0.430380
6         8.0  0.449367
7         9.0  0.457806
8        10.0  0.432489
9        11.0  0.459916
10       12.0  0.556962
11       13.0  0.449367
12       14.0  0.561181


In [28]:
# Combine feature names and coefficients into pandas DataFrame
feature_names = pd.DataFrame(train_X.columns, columns = ['Feature'])
log_coef = pd.DataFrame(np.transpose(logreg.coef_), columns = ['Coefficient'])
coefficients = pd.concat([feature_names, log_coef], axis = 1)

# Calculate exponent of the logistic regression coefficients
coefficients['Exp_Coefficient'] = np.exp(coefficients['Coefficient'])

# Remove coefficients that are equal to zero
coefficients = coefficients[coefficients['Coefficient']!=0]

# Print the values sorted by the exponent coefficient
print(coefficients.sort_values(by=['Exp_Coefficient']))

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [29]:
# Export graphviz object from the trained decision tree 
exported = tree.export_graphviz(decision_tree=mytree, 
			# Assign feature names
            out_file=None, feature_names=train_X.columns, 
			# Set precision to 1 and add class names
			precision=1, class_names=['Not churn','Churn'], filled = True)

# Call the Source function and pass the exported graphviz object
graph = graphviz.Source(exported)

# Display the decision tree
display_image("/usr/local/share/datasets/decision_tree_rules.png")

AttributeError: 'numpy.ndarray' object has no attribute 'columns'